[Official documentation on chat from LangChain](https://python.langchain.com/en/latest/modules/models/chat/getting_started.html)

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)


# Supporting libraries
import os
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
# OPENAI_API_KEY = os.getenv('OPENAI_API_KEY', 'YourAPIKeyIfNotSet')

### 0. Simple Input/Output Still works

You just need to pass it as a message instead. More on this in a second.

In [3]:
chat = ChatOpenAI()

In [4]:
message = [HumanMessage(content="What is the name of the most populous state in the USA?")]

chat(message)

AIMessage(content='The most populous state in the USA is California.', additional_kwargs={}, example=False)

### 1. Chat Messages

* HumanMessage: A message sent from the perspective of the human
* AIMessage: A message sent from the perspective of the AI the human is interacting with
* SystemMessage: A message setting the objectives the AI should follow
* ChatMessage: A message allowing for arbitrary setting of role. You won’t be using this too much

In [5]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming.")
]
chat(messages)

AIMessage(content='I hate programming.', additional_kwargs={}, example=False)

In [6]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming."),
    AIMessage(content='I hate programming.'),
    HumanMessage(content="The moon is out")
]
chat(messages)

AIMessage(content='The moon is not out.', additional_kwargs={}, example=False)

In [7]:
messages = [
    SystemMessage(content="Say the opposite of what the user says"),
    HumanMessage(content="I love programming."),
    AIMessage(content='I hate programming.'),
    HumanMessage(content="What is the first thing that I said?")
]
chat(messages)

AIMessage(content='What is the last thing that I said?', additional_kwargs={}, example=False)

#### Batch Messages

In [8]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful word machine that creates an alliteration using a base word"),
        HumanMessage(content="Base word: Apple")
    ],
    [
        SystemMessage(content="You are a helpful word machine that creates an alliteration using a base word"),
        HumanMessage(content="Base word: Dog")
    ],
]
chat.generate(batch_messages)

LLMResult(generations=[[ChatGeneration(text='Astonishingly appetizing apple', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='Astonishingly appetizing apple', additional_kwargs={}, example=False))], [ChatGeneration(text='Delightful dog dashes through the daisies.', generation_info={'finish_reason': 'stop'}, message=AIMessage(content='Delightful dog dashes through the daisies.', additional_kwargs={}, example=False))]], llm_output={'token_usage': {'prompt_tokens': 60, 'completion_tokens': 18, 'total_tokens': 78}, 'model_name': 'gpt-3.5-turbo'}, run=[RunInfo(run_id=UUID('f3f8684e-991a-42d4-bdb4-547f162f2bdc')), RunInfo(run_id=UUID('d7f7a93d-9cb1-4087-ba36-3ebcd40c74f2'))])

### Prompt Templates
With one or more `MessagePromptTemplates` you can build a `ChatPromptTemplate`

In [9]:
# Make SystemMessagePromptTemplate
prompt=PromptTemplate(
    template="Propose creative ways to incorporate {food_1} and {food_2} in the cuisine of the users choice.",
    input_variables=["food_1", "food_2"]
)

system_message_prompt = SystemMessagePromptTemplate(prompt=prompt)

In [10]:
# Output of system_message_prompt
system_message_prompt.format(food_1="Bacon", food_2="Shrimp")

SystemMessage(content='Propose creative ways to incorporate Bacon and Shrimp in the cuisine of the users choice.', additional_kwargs={})

In [11]:
# Make HumanMessagePromptTemplate
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [12]:
# Create ChatPromptTemplate: Combine System + Human
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['food_1', 'food_2', 'text'], output_parser=None, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['food_1', 'food_2'], output_parser=None, partial_variables={}, template='Propose creative ways to incorporate {food_1} and {food_2} in the cuisine of the users choice.', template_format='f-string', validate_template=True), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='{text}', template_format='f-string', validate_template=True), additional_kwargs={})])

In [13]:
chat_prompt_with_values = chat_prompt.format_prompt(food_1="Bacon", \
                                                   food_2="Shrimp", \
                                                   text="I really like food from Germany.")

chat_prompt_with_values.to_messages()

[SystemMessage(content='Propose creative ways to incorporate Bacon and Shrimp in the cuisine of the users choice.', additional_kwargs={}),
 HumanMessage(content='I really like food from Germany.', additional_kwargs={}, example=False)]

In [14]:
response = chat(chat_prompt_with_values.to_messages()).content
print (response)

Incorporating bacon and shrimp into German cuisine can add a unique twist to traditional dishes. Here are a few creative ideas:

1. Bacon-Wrapped Shrimp Schnitzel: Prepare a classic German schnitzel by pounding thin slices of pork or chicken, then wrap each piece with a strip of bacon. Bread and fry the schnitzels until golden brown and serve them with a side of tartar sauce or lemon wedges.

2. Shrimp and Bacon Sauerkraut Rolls: Take sauerkraut and mix it with cooked shrimp and crispy bacon. Roll the mixture in thinly sliced bacon and secure with toothpicks. Bake until the bacon is crispy and serve it as an appetizer or a side dish.

3. Bacon and Shrimp Potato Salad: Enhance the traditional German potato salad by adding cooked shrimp and crispy bacon bits. Toss the ingredients with a tangy mustard and vinegar dressing for a flavorful twist on a classic dish.

4. Bacon-Wrapped Shrimp Bratwurst: Create a unique fusion by wrapping shrimp with bacon and grilling them alongside bratwurst s

#### With Streaming

In [16]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
chat = ChatOpenAI(streaming=True, callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]), verbose=True, temperature=0)

resp = chat(chat_prompt_with_values.to_messages())

Incorporating bacon and shrimp into German cuisine can add a delicious twist to traditional dishes. Here are a few creative ideas:

1. Bacon-Wrapped Shrimp Schnitzel: Prepare a classic schnitzel by pounding a pork cutlet, then wrap it with bacon before breading and frying. Serve it with a side of shrimp scampi, where the shrimp is sautéed in garlic, butter, and white wine.

2. Bacon and Shrimp Sauerkraut Rolls: Stuff large shrimp with a mixture of sauerkraut, bacon, and spices. Wrap each shrimp with a slice of bacon and secure with toothpicks. Bake until the bacon is crispy and the shrimp is cooked through. Serve with a side of warm German potato salad.

3. Shrimp and Bacon Spätzle: Add cooked shrimp and crispy bacon to traditional German spätzle. Sauté the shrimp and bacon together, then toss with cooked spätzle, butter, fresh herbs, and a squeeze of lemon juice. Garnish with grated Parmesan cheese.

4. Bacon-Wrapped Shrimp Bratwurst: Wrap jumbo shrimp with bacon and grill them alongs